In [1]:
%pip install lxml polars


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: /Users/pletcher/code/writing/articles/2024-11-28_tragedy-dfs/.venv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import polars as pl

df = pl.read_parquet("./greek-tragedy-by-line.parquet")
df

n,urn,dramatist,title,speaker,text,tokens
str,str,str,str,str,str,list[str]
"""1""","""urn:cts:greekLit:tlg0006.tlg01…","""Euripides""","""Ἰφιγένεια ἐν Ταύροις""","""Ἰφιγένεια""","""Πέλοψ ὁ Ταντάλειος ἐς Πῖσαν μο…","[""πελοψ"", ""ο"", … ""μολων""]"
"""2""","""urn:cts:greekLit:tlg0006.tlg01…","""Euripides""","""Ἰφιγένεια ἐν Ταύροις""","""Ἰφιγένεια""","""θοαῖσιν ἵπποις Οἰνομάου γαμεῖ …","[""θοαισιν"", ""ιπποισ"", … ""κορην""]"
"""3""","""urn:cts:greekLit:tlg0006.tlg01…","""Euripides""","""Ἰφιγένεια ἐν Ταύροις""","""Ἰφιγένεια""","""ἐξ ἧς Ἀτρεὺς ἔβλαστεν· Ἀτρέως …","[""εξ"", ""ησ"", … ""παισ""]"
"""4""","""urn:cts:greekLit:tlg0006.tlg01…","""Euripides""","""Ἰφιγένεια ἐν Ταύροις""","""Ἰφιγένεια""","""Μενέλαος Ἀγαμέμνων τε· τοῦ δʼ …","[""μενελαοσ"", ""αγαμεμνων"", … ""εγω""]"
"""5""","""urn:cts:greekLit:tlg0006.tlg01…","""Euripides""","""Ἰφιγένεια ἐν Ταύροις""","""Ἰφιγένεια""","""τῆς Τυνδαρείας θυγατρὸς Ἰφιγέν…","[""τησ"", ""τυνδαρειασ"", … ""παισ""]"
…,…,…,…,…,…,…
"""1526""","""urn:cts:greekLit:tlg0011.tlg00…","""Sophocles""","""Οἰδίπους Τύραννος""","""Χορός""","""οὗ τίς οὐ ζήλῳ πολιτῶν ἦν τύχα…","[""ου"", ""τισ"", … ""επιβλεπων""]"
"""1527""","""urn:cts:greekLit:tlg0011.tlg00…","""Sophocles""","""Οἰδίπους Τύραννος""","""Χορός""","""εἰς ὅσον κλύδωνα δεινῆς συμφορ…","[""εισ"", ""οσον"", … ""εληλυθεν""]"
"""1528""","""urn:cts:greekLit:tlg0011.tlg00…","""Sophocles""","""Οἰδίπους Τύραννος""","""Χορός""","""ὥστε θνητὸν ὄντα κείνην τὴν τε…","[""ωστε"", ""θνητον"", … ""ιδειν""]"


In [8]:
def ttr_by_play():
    with pl.Config(tbl_rows=-1):
        res = (
            df.group_by("dramatist", "title")
            .agg(pl.col("tokens").explode())
            .select(
                "dramatist",
                "title",
                n_types=pl.col("tokens").list.n_unique(),
                n_tokens=pl.col("tokens").list.len(),
                ttr=(pl.col("tokens").list.n_unique() / pl.col("tokens").list.len()),
            )
            .sort("ttr")
        )

        return res
    
def ttr_by_dramatist():
    res = ttr_by_play()
    avgs = (
        res.group_by("dramatist")
        .agg(pl.col("ttr").mean().alias("avg_ttr"))
        .select("dramatist", "avg_ttr")
        .sort("avg_ttr")
    )

    return avgs

In [ ]:
def write_speakers_by_dramatist_and_play():
    drama_speakers = (
        df.select("dramatist", "title", "speaker")
        .group_by("dramatist", "title")
        .agg(pl.col("speaker").unique())
    )

    with open("speakers.csv", "w+") as f:
        f.write("dramatist,title,speaker,gender\n")

        for dramatist, title, speakers in drama_speakers.iter_rows():
            for speaker in speakers:
                f.write(f"{dramatist},{title},{speaker},\n")

shape: (34, 3)
┌───────────┬──────────────────────┬─────────────────────────────────┐
│ dramatist ┆ title                ┆ speaker                         │
│ ---       ┆ ---                  ┆ ---                             │
│ str       ┆ str                  ┆ list[str]                       │
╞═══════════╪══════════════════════╪═════════════════════════════════╡
│ Aeschylus ┆ Ἑπτὰ ἐπὶ Θήβας       ┆ ["Ἡμιχόριον Α", "Ἄγγελος", … "… │
│ Euripides ┆ Ἰφιγένεια ἐν Ταύροις ┆ ["Ὀρέστης", "Βουκόλος", … "Χορ… │
│ Euripides ┆ Ὀρέστης              ┆ ["Πυλάδης", "Χορός", … "Ὀρέστη… │
│ Euripides ┆ Ἱκέτιδες             ┆ ["Παῖδες", "Ἄδραστος", … "Αἴθρ… │
│ Euripides ┆ Βάκχαι               ┆ ["Διονυσος", "Ἄγγελος", … "Ἀγα… │
│ …         ┆ …                    ┆ …                               │
│ Sophocles ┆ Οἰδίπους Τύραννος    ┆ ["Ἄγγελος", "Κρέων", … "Χορός"… │
│ Aeschylus ┆ Χοηφóρoι             ┆ ["Τροφός", "Αἴγισθος", … "Ὀρέσ… │
│ Sophocles ┆ Οἰδίπους ἐπὶ Κολωνῷ  ┆ ["Ξένος", "Κρέων", … "Ἰσμ